In [45]:
import numpy as np 
import pandas as pd
import glob
import cv2
import os
from pathlib import Path
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.models import Sequential

In [46]:
import warnings
warnings.filterwarnings("ignore")

In [47]:
print(os.listdir(r'C:\Users\Abhishek\JUPYTER\Detecting malaria'))

['cell-images-for-detecting-malaria.zip', 'cell_images']


In [48]:
directory = Path('C:/Users/Abhishek/JUPYTER/Detecting malaria/cell_images')

Parasitized_dir = directory / 'Parasitized'
Uninfected_dir  = directory / 'Uninfected'

In [49]:
X = []
Y = []

In [50]:
# DATA CLEANSING
Parasitized  = Parasitized_dir.glob('*.png')
Uninfected   = Uninfected_dir.glob('*.png')

In [51]:
height = 64
width  = 64
for u in Uninfected:
    image = cv2.imread(str(u))
    resizeimage = cv2.resize(image, (height,width))
    img = resizeimage.astype(np.float32)/255.
    label = to_categorical(0, num_classes=2)
    X.append((img))
    Y.append((label))
    
for p in Parasitized:
    image = cv2.imread(str(p))
    resizeimage = cv2.resize(image, (height,width))
    img = resizeimage.astype(np.float32)/255.
    label = to_categorical(1, num_classes=2)
    X.append((img))
    Y.append((label))
    
X = np.array(X)
Y = np.array(Y)

In [52]:
X.shape , Y.shape

((27558, 64, 64, 3), (27558, 2))

In [53]:
X , Y = shuffle(X , Y , random_state = 42)

In [54]:
X_train , X_test , Y_train , Y_test = train_test_split(X , Y , test_size = 0.2 , random_state = 0)

In [55]:
model = Sequential()

model.add(Conv2D(16,(3,3),   padding='same' ,activation="relu" ,input_shape=(64,64,3)))
model.add(Conv2D(16,(3,3),  padding='same' ,activation="relu"))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(32,(3,3),  padding='same' ,activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(BatchNormalization())
          
model.add(Conv2D(32,(3,3), padding='same' ,activation="relu"))
model.add(BatchNormalization())
          
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2,activation="softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 64, 64, 16)        448       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 64, 16)        2320      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 32)        4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 32)        9248      
__________

In [56]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [57]:
datagen = ImageDataGenerator(
    zoom_range = 0.3 ,
    rotation_range=20,
    width_shift_range=0.2,
    horizontal_flip=True,
    fill_mode = 'nearest')


datagen.fit(X)

In [58]:
model.fit_generator(datagen.flow(X_train , Y_train, batch_size=64),
                    steps_per_epoch=len(X) / 64, 
                    epochs=20,
                    validation_data=(X_test , Y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/20
431/430 [==============================] - 255s 593ms/step - loss: 0.5210 - acc: 0.7563 - val_loss: 0.5123 - val_acc: 0.8128
Epoch 2/20
431/430 [==============================] - 163s 379ms/step - loss: 0.2426 - acc: 0.9179 - val_loss: 0.5228 - val_acc: 0.6756
Epoch 3/20
431/430 [==============================] - 174s 403ms/step - loss: 0.2135 - acc: 0.9292 - val_loss: 0.1684 - val_acc: 0.9476
Epoch 4/20
431/430 [==============================] - 166s 384ms/step - loss: 0.1958 - acc: 0.9349 - val_loss: 0.1474 - val_acc: 0.9554
Epoch 5/20
431/430 [==============================] - 168s 391ms/step - loss: 0.1885 - acc: 0.9366 - val_loss: 0.1515 - val_acc: 0.9492
Epoch 6/20
431/430 [==============================] - 171s 396ms/step - loss: 0.1807 - acc: 0.9390 - val_loss: 0.6025 - val_acc: 0.8759
Epoch 7/20
431/430 [==============================] - 173s 402ms/step - loss: 0.1846 - acc

In [59]:
loss, test_accuracy = model.evaluate(X_test, Y_test)
print(loss)
print(test_accuracy)

5512/5512 [==============================] - 12s 2ms/step
0.12189931378947201
0.9622641509433962
